<a href="https://colab.research.google.com/github/maxmatical/fast.ai/blob/master/fastai_%2B_blurr_%2B_deberta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install --upgrade fastai
!pip install transformers
!pip install ohmeow-blurr
!pip install bentoml

In [2]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.7.10
fastai        : 2.2.7
fastcore      : 1.3.19
fastprogress  : 0.2.7
torch         : 1.7.1
nvidia driver : 460.32
torch cuda    : 10.2 / is available
torch cudnn   : 7605 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : Tesla K80

=== Environment === 
platform      : Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Thu Jul 23 08:00:38 PDT 2020
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once i

In [3]:
from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [5]:
task = HF_TASKS_AUTO.SequenceClassification

pretrained_model_name = 'microsoft/deberta-base'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,  task=task)

In [6]:
# single input
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = dblock.dataloaders(imdb_df, bs=4)

In [7]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"This film sat on my Tivo for weeks before I watched it. I dreaded a self-indulgent yuppie flick about relationships gone bad. I was wrong; this was an engrossing excursion into the screwed-up libidos of New Yorkers.<br /><br />The format is the same as Max Ophuls' ""La Ronde,"" based on a play by Arthur Schnitzler, who is given an ""inspired by"" credit. It starts from one person, a prostitute, standing on a street corner in Brooklyn. She is picked up by a home contractor, who has sex with her on the hood of a car, but can't come. He refuses to pay her. When he's off peeing, she answers his cell phone and takes a message. She runs away with his keys.<br /><br />Then the story switches to the contractor, who pays a professional call on a rich, bored New York woman, who plays with him until he is aroused, then she pulls away. She tells him how desperate and unhappy she is; he tells her how beautiful she is, and lucky. As he is leaving, she asks if he would have sex with her. She sits on top of him, bounces up and down. This time he comes, the he leaves.<br /><br />The woman and her husband throw a dinner party for their trendy friends. Hubby (Robert) is talking business, wife (Ellen) is bored, and switches the subject to sex, and how often men and women think about it. Husband switches conversation to desert. Later, after the guests leave, Ellen tries to entice Robert into sex. Robert wants none of it, and puts on a jazz record. Ellen turns on the radio; Robert turns up the music; Ellen turns on the TV; Robert turns on another TV. Cacophony ensues. Ellen goes up on the roof, Robert joins her. Ellen confesses that she needs to experience more men, men other than Robert. Robert says that he too needs to experi

In [8]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=LabelSmoothingCrossEntropyFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="accuracy", fname="model_1")
]



In [ ]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)

# Inference and deployment with BentoML

Notess:
- can just treat as a fastai model and use `test_dl` and `get_preds` to make inference
- Can also quantize the underpying pytorch model with `learn.model.hf_model`

In [10]:
learn.validate()

epoch,train_loss,valid_loss,accuracy,time


(#2) [0.6920732259750366,0.5350000262260437]

In [14]:
test_dl = learn.dls.test_dl(["this movie sucks", "i really like this movie"])
learn.get_preds(dl=test_dl, with_decoded=True)

(tensor([[0.5109, 0.4891],
         [0.5114, 0.4886]]), None, tensor([0, 0]))

In [27]:
preds = learn.get_preds(dl=test_dl, with_decoded=True)[-1].cpu().numpy()
preds

array([0, 0])

### Using only the underlying pytorch model

allows for `torchscript` with jit 

In [ ]:
torch.save(learn.model.hf_model, "deberta.pt")
inf_model = torch.load("deberta.pt", map_location=torch.device("cpu"))
inf_model.eval()

In [37]:
text = "This movie sucks"
text_ids = hf_tokenizer.encode(text)
inp = tensor(text_ids)[None]

inf_model(inp)

SequenceClassifierOutput([('logits',
                           tensor([[0.0442, 0.0002]], grad_fn=<AddmmBackward>))])

In [47]:
texts = ["This movie sucks", "i really like this movie"]
text_ids = hf_tokenizer.batch_encode_plus(texts, 
                                          padding='longest',
                                          truncation=True,
                                          max_length=256,
                                          return_tensors="pt")

text_ids

{'input_ids': tensor([[    1,   713,  1569, 29384,     2,     0,     0],
        [    1,   118,   269,   101,    42,  1569,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

In [48]:
# creating pytorch dataloader
from torch.utils.data import TensorDataset, BatchSampler
input_ids_inf = text_ids['input_ids']
attention_masks_inf = text_ids['attention_mask']

dataset_inf = TensorDataset(input_ids_inf, attention_masks_inf)
dataloader_inf = DataLoader(dataset_inf, sampler=SequentialSampler(dataset_inf), batch_size=32)

In [61]:
device = "cpu"
all_logits = np.empty([0,2])

for batch in dataloader_inf:

    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              }
    with torch.no_grad():        
        outputs = inf_model(**inputs)

    logits = outputs[0]

    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])

In [53]:
all_logits

array([[0.51101547, 0.48898458],
       [0.51140863, 0.48859134]])

Now we can deploy with BentoML using either fastai or pytorch frameworks